
# UC Toolkit: Tool Functions

In [0]:
from databricks.sdk import WorkspaceClient

ws = WorkspaceClient()
current_user = ws.current_user.me().user_name
first_name, last_name = current_user.split('@')[0].split('.')
formatted_name = f"{first_name[0]}_{last_name}"

catalog = f'dbdemos_{formatted_name}'
schema = 'chem_manufacturing'
print(f"Catalog name: {catalog}")

In [0]:
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")


## Vector Store Search Functions

To do similarity search the Databricks Vector Index can be used with SQL Function VECTOR_SEARCH() for convinence. 


### Find Similar Products

In [0]:
%sql
-- Function to find similar products using vector search
CREATE OR REPLACE FUNCTION find_similar_products(product_description STRING)
RETURNS TABLE (product_id STRING, full_description STRING, product_name STRING, category STRING, molecular_weight DOUBLE, density DOUBLE, melting_point DOUBLE, boiling_point DOUBLE, chemical_formula STRING, search_score DOUBLE, price_per_unit DOUBLE)
LANGUAGE SQL
COMMENT 'Find products with similarity search based on description of product name, application areas, storage_conditions. This helps customers find products and alternatives. This query will return details like product_id, full_description, product_name, category, molecular_weight, density, melting_point, boiling_point, chemical_formula, search_score , price_per_unit'
RETURN
  SELECT product_id, full_description, product_name, category, molecular_weight, density, melting_point, boiling_point, chemical_formula, search_score , price_per_unit
  FROM VECTOR_SEARCH(
    index => 'dbdemos_a_jack.chem_manufacturing.products_index',
    query => product_description,
    num_results => 2
  )
  ORDER BY search_score DESC 


In [0]:
%sql
-- let's test our function:
SELECT * FROM find_similar_products('A product that has a name similar to "Synth" and is for the food industry');


### Find safety protocolls and research notes 

In [0]:
%sql
-- Function to find safety protocols by description
CREATE OR REPLACE FUNCTION find_safety_protocols(search_text STRING)
RETURNS TABLE (description_id STRING, description_type STRING, product_id STRING, title STRING, content STRING, search_score DOUBLE)
LANGUAGE SQL
COMMENT 'Find safety, procedures, research protocools with similarity search for chemicals matching the description. Returns relevant safety information for handling chemicals description_id, description_type, product_id, title, content, search_score.'
RETURN
  SELECT description_id, description_type, product_id, title, content, search_score
  FROM VECTOR_SEARCH(
    index => 'dbdemos_a_jack.chem_manufacturing.descriptions_index',
    query => search_text,
    num_results => 2
  )
  ORDER BY search_score DESC 

In [0]:
%sql
-- let's test our function:
SELECT * FROM find_safety_protocols('Find protocools about impurity reduction');


## SQL Functions

In [0]:
%sql
-- Function to get detailed reaction information
CREATE OR REPLACE FUNCTION get_product(productid STRING)
RETURNS TABLE (product_id string, product_name string, category string,
chemical_formula string, molecular_weight double, density double, melting_point double, boiling_point double,description string, application_areas string, storage_conditions string, full_description string, creation_date string, price_per_unit double
)
LANGUAGE SQL
COMMENT 'Retrieve detailed information about a product with product id. Returns product_id, product_name, category, chemical_formula, molecular_weight, density, melting_point, boiling_point, description, application_areas, storage_conditions, full_description, creation_date, price_per_unit'
RETURN
  SELECT *
  FROM dbdemos_a_jack.chem_manufacturing.products
  WHERE product_id = productid;

In [0]:
%sql
SELECT * FROM get_product('P0001')

In [0]:
%sql
-- Function to get detailed reaction information
CREATE OR REPLACE FUNCTION get_safety_protocols(productid STRING)
RETURNS TABLE (description_id STRING, description_type STRING, product_id STRING, title STRING, content STRING)
LANGUAGE SQL
COMMENT 'Get safety, procedures, research protocools with product id for chemicals matching the description. Returns relevant safety information for handling chemicals description_id, description_type, product_id, title, content'
RETURN
  SELECT description_id, description_type, product_id, title, content
  FROM dbdemos_a_jack.chem_manufacturing.descriptions
  WHERE product_id = productid;

In [0]:
%sql
SELECT * FROM get_safety_protocols('P0001')

In [0]:
%sql
-- Function to get detailed reaction information
CREATE OR REPLACE FUNCTION get_reaction_details(productid STRING)
RETURNS TABLE (reaction_id STRING, reaction_name STRING, reaction_type STRING, catalyst STRING, solvent STRING, temperature DOUBLE, pressure DOUBLE, reaction_time DOUBLE, energy_consumption DOUBLE, hazards STRING)
LANGUAGE SQL
COMMENT 'Retrieve detailed information with product id about chemical reactions used to produce a specific product. Returns reaction conditions, reaction type, catalyst name, solvent, temperature, pressure, reaction time, energy consumption, hazards.'
RETURN
  SELECT reaction_id, reaction_name, reaction_type, catalyst, solvent, temperature, pressure, reaction_time, energy_consumption, hazards
  FROM dbdemos_a_jack.chem_manufacturing.reactions
  WHERE product_id = productid;

In [0]:
%sql
SELECT * FROM get_reaction_details('P0001')

In [0]:
%sql
-- Function to analyze product quality metrics
CREATE OR REPLACE FUNCTION analyze_product_quality(productid STRING)
RETURNS TABLE (product_name STRING, total_tests INT, passed_tests INT, failed_tests INT, pass_rate DOUBLE)
LANGUAGE SQL
COMMENT 'Analyze quality metrics for a specific product. Shows test results, pass rates, and common quality issues.'
RETURN
  SELECT 
    p.product_name,
    COUNT(q.test_id) as total_tests,
    SUM(CASE WHEN q.test_result = 'Pass' THEN 1 ELSE 0 END) as passed_tests,
    SUM(CASE WHEN q.test_result = 'Fail' THEN 1 ELSE 0 END) as failed_tests,
    ROUND(SUM(CASE WHEN q.test_result = 'Pass' THEN 1 ELSE 0 END) * 100.0 / COUNT(q.test_id), 2) as pass_rate
  FROM dbdemos_a_jack.chem_manufacturing.quality_control q
  JOIN dbdemos_a_jack.chem_manufacturing.batches b ON q.batch_id = b.batch_id
  JOIN dbdemos_a_jack.chem_manufacturing.products p ON b.product_id = p.product_id
  WHERE b.product_id = productid
  GROUP BY p.product_name;

In [0]:
%sql
SELECT * FROM analyze_product_quality('P0002')


## Python Functions



###Unit convertion tool: Supporting common unit conversion
Add a function to allow chemical unit conversions.

Databricks runs the python in a safe container. However, we'll filter what the function can do to avoid any potential issues with prompt injection (so that the user cannot execute other python instructions).

In [0]:
%sql
CREATE OR REPLACE FUNCTION convert_chemical_unit(value DOUBLE, from_unit STRING, to_unit STRING, mol_weight DOUBLE)
RETURNS DOUBLE
LANGUAGE PYTHON
COMMENT 'Convert between different chemical measurement units (g, kg, mol, L, mL) to units (kg, g, mL, L, mol) usage: convert_chemical_unit(1, "kg", "g", 0) if mol is not provided use 0'
AS
$$
  unit_conversions = {
    'g_to_kg': lambda x: x / 1000,
    'kg_to_g': lambda x: x * 1000,
    'L_to_mL': lambda x: x * 1000,
    'mL_to_L': lambda x: x / 1000,
    'g_to_mol': lambda x, mw: x / mw if mw else None,
    'mol_to_g': lambda x, mw: x * mw if mw else None
  }
  
  conversion_key = f"{from_unit.lower()}_to_{to_unit.lower()}"
  
  if conversion_key in unit_conversions:
    if conversion_key in ['g_to_mol', 'mol_to_g']:
      if mol_weight is None:
        return f"Molecular weight required for {conversion_key} conversion"
      return unit_conversions[conversion_key](value, mol_weight)
    else:
      return unit_conversions[conversion_key](value)
  else:
    return f"Conversion from {from_unit} to {to_unit} not supported"
$$;

In [0]:
%sql
SELECT convert_chemical_unit(1, "g", "mol", 0.58);

### Calculator tool: Supporting math operations 
Let's add a function to allow our LLM to execute any Math operation. 

Databricks runs the python in a safe container. However, we'll filter what the function can do to avoid any potential issues with prompt injection (so that the user cannot execute other python instructions).

In [0]:
%sql
CREATE OR REPLACE FUNCTION compute_math(expr STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Run any mathematical function and returns the result as output. Supports python syntax like math.sqrt(13)'
AS
$$
  import ast
  import operator
  import math
  operators = {ast.Add: operator.add, ast.Sub: operator.sub, ast.Mult: operator.mul, ast.Div: operator.truediv, ast.Pow: operator.pow, ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv, ast.UAdd: operator.pos, ast.USub: operator.neg}
    
  # Supported functions from the math module
  functions = {name: getattr(math, name) for name in dir(math) if callable(getattr(math, name))}

  def eval_node(node):
    if isinstance(node, ast.Num):  # <number>
      return node.n
    elif isinstance(node, ast.BinOp):  # <left> <operator> <right>
      return operators[type(node.op)](eval_node(node.left), eval_node(node.right))
    elif isinstance(node, ast.UnaryOp):  # <operator> <operand> e.g., -1
      return operators[type(node.op)](eval_node(node.operand))
    elif isinstance(node, ast.Call):  # <func>(<args>)
      func = node.func.id
      if func in functions:
        args = [eval_node(arg) for arg in node.args]
        return functions[func](*args)
      else:
        raise TypeError(f"Unsupported function: {func}")
    else:
      raise TypeError(f"Unsupported type: {type(node)}")  
  try:
    if expr.startswith('```') and expr.endswith('```'):
      expr = expr[3:-3].strip()      
    node = ast.parse(expr, mode='eval').body
    return eval_node(node)
  except Exception as ex:
    return str(ex)
$$;
-- let's test our function:
SELECT compute_math("(2+2)/3") as result;

### Executing a Python function to fetch external dataset in realtime: getting the weather

We want our stylist assistant to give us recommendations based on the weather. Let's add a tool to fetch the weather based on longitude/latitude, using Python to call an external Weather API.

**Note: This will be run by a serverless compute, and accessing external data, therefore requires serverless network egress access. If this fails in a serverless setup/playground, make sure you are allowing it in your networking configuration (open the workspace network option at admin account level)**

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_weather(latitude DOUBLE, longitude DOUBLE)
RETURNS STRUCT<temperature_in_celsius DOUBLE, rain_in_mm DOUBLE>
LANGUAGE PYTHON
COMMENT 'This function retrieves the current temperature and rain information for a given latitude and longitude using the Open-Meteo API.'
AS
$$
  try:
    import requests as r
    #Note: this is provided for education only, non commercial - please get a license for real usage: https://api.open-meteo.com. Let s comment it to avoid issues for now
    #weather = r.get(f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,rain&forecast_days=1').json()
    return {
      "temperature_in_celsius": weather["current"]["temperature_2m"],
      "rain_in_mm": weather["current"]["rain"]
    }
  except:
    return {"temperature_in_celsius": 22.0, "rain_in_mm": 0.0}
$$;

-- let's test our function:
SELECT get_weather(52.52, 13.41) as weather;

### Run any python function

If we are creating a coding assistant, it can be useful to allow our AI System to run python code to try them and output the results to the user. **Be careful doing that, as any code could be executed by the user.**

Here is an example:

In [0]:
%sql
CREATE OR REPLACE FUNCTION execute_python_code(python_code STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT "Run python code. The code should end with a return statement and this function will return it as a string. Only send valid python to this function. Here is an exampe of python code input: 'def square_function(number):\\n  return number*number\\n\\nreturn square_function(3)'"
AS
$$
    import traceback
    try:
        import re
        # Remove code block markers (e.g., ```python) and strip whitespace```
        python_code = re.sub(r"^\s*```(?:python)?|```\s*$", "", python_code).strip()
        # Unescape any escaped newline characters
        python_code = python_code.replace("\\n", "\n")
        # Properly indent the code for wrapping
        indented_code = "\n    ".join(python_code.split("\n"))
        # Define a wrapper function to execute the code
        exec_globals = {}
        exec_locals = {}
        wrapper_code = "def _temp_function():\n    "+indented_code
        exec(wrapper_code, exec_globals, exec_locals)
        # Execute the wrapped function and return its output
        result = exec_locals["_temp_function"]()
        return result
    except Exception as ex:
        return traceback.format_exc()
$$;
-- let's test our function:

SELECT execute_python_code("return 'Hello! '* 3") as result;

## LLM-Based Functions

With ai_query we can apply LLM prompts on each row of a table. This can be very practical for tasks that would otherwise require tedious 1:n comparisons. 

In [0]:
%sql
-- Function to recommend product alternatives
CREATE OR REPLACE FUNCTION recommend_product_alternatives(product_name STRING, reason STRING)
RETURNS STRING
LANGUAGE SQL
COMMENT 'Recommends alternative products based on the specified reason (cost, availability, performance, safety)'
RETURN SELECT ai_query('databricks-meta-llama-3-70b-instruct',
    CONCAT(
      "You are a chemical product specialist. A customer is looking for alternatives to product ", 
      product_name, " for the following reason: ", reason, 
      "Compare the product with the input product briefly on the mentioned criterias"
    ) || product_id 
  ) AS alternative_option
  FROM dbdemos_a_jack.chem_manufacturing.products;

In [0]:
SELECT * FROM recommend_product_alternatives("ProCat-Z91", "to expensive")